# Persiapan Data dan Environment

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, classification_report
import pickle
import os

In [15]:
(x_train_full, y_train_full), (x_test, y_test) = keras.datasets.cifar10.load_data()

x_train_full = x_train_full.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

split_idx = 40000
x_train = x_train_full[:split_idx]
y_train = y_train_full[:split_idx]
x_val = x_train_full[split_idx:]
y_val = y_train_full[split_idx:]

y_train = y_train.flatten()
y_val = y_val.flatten()
y_test = y_test.flatten()

print(f"Training data: {x_train.shape}, {y_train.shape}")
print(f"Validation data: {x_val.shape}, {y_val.shape}")
print(f"Test data: {x_test.shape}, {y_test.shape}")

Training data: (40000, 32, 32, 3), (40000,)
Validation data: (10000, 32, 32, 3), (10000,)
Test data: (10000, 32, 32, 3), (10000,)


# Implementasi Model CNN Baseline

In [16]:
def create_cnn_model(conv_layers=2, filters_per_layer=[32, 64], 
                     kernel_sizes=[3, 3], pooling_type='max'):
    model = keras.Sequential()
    
    model.add(layers.Input(shape=(32, 32, 3)))
    
    for i in range(conv_layers):
        filters = filters_per_layer[i] if i < len(filters_per_layer) else filters_per_layer[-1]
        kernel_size = kernel_sizes[i] if i < len(kernel_sizes) else kernel_sizes[-1]
        
        model.add(layers.Conv2D(filters, kernel_size, activation='relu', padding='same'))
        
        if pooling_type == 'max':
            model.add(layers.MaxPooling2D(2, 2))
        elif pooling_type == 'avg':
            model.add(layers.AveragePooling2D(2, 2))
    
    model.add(layers.Flatten())
    
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation='softmax'))
    
    return model

In [17]:
def plot_training_history(histories, labels, title, experiment_name="cnn_experiment"): # Tambahkan experiment_name
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 2, 1)
    for history_dict, label in zip(histories, labels): # Asumsi histories adalah list of dicts
        plt.plot(history_dict['loss'], label=f'{label} - Training')
        plt.plot(history_dict['val_loss'], label=f'{label} - Validation', linestyle='--')
    plt.title(f'{title} - Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    for history_dict, label in zip(histories, labels): # Asumsi histories adalah list of dicts
        plt.plot(history_dict['accuracy'], label=f'{label} - Training')
        plt.plot(history_dict['val_accuracy'], label=f'{label} - Validation', linestyle='--')
    plt.title(f'{title} - Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()

    # Bagian untuk menyimpan plot
    plot_filename = f"results/cnn_plots/{experiment_name}_{title.replace(' ', '_')}_history.png"
    os.makedirs(os.path.dirname(plot_filename), exist_ok=True) # Membuat direktori jika belum ada
    plt.savefig(plot_filename)
    print(f"Saved plot to {plot_filename}")
    plt.show() # Anda masih bisa menampilkannya jika mau, atau plt.close() jika tidak ingin ditampilkan langsung
    
def plot_and_save_single_history(history_obj, model_name_str, experiment_label="cnn_experiment"):
    history_data = history_obj.history # history_obj adalah objek History dari Keras
    plt.figure(figsize=(12, 4))

    # Plot Loss
    plt.subplot(1, 2, 1)
    plt.plot(history_data['loss'], label='Training Loss')
    plt.plot(history_data['val_loss'], label='Validation Loss')
    plt.title(f'Loss - {model_name_str}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history_data['accuracy'], label='Training Accuracy')
    plt.plot(history_data['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Accuracy - {model_name_str}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()

    plot_filename = f"results/cnn_plots/{experiment_label}_{model_name_str.replace(' ', '_').replace('/', '_')}_history.png"
    os.makedirs(os.path.dirname(plot_filename), exist_ok=True)
    plt.savefig(plot_filename)
    print(f"Saved plot to {plot_filename}")
    plt.close() # Tutup plot agar tidak menumpuk jika dijalankan dalam loop

In [18]:
def train_model(model, model_name, epochs=2): # model_name di sini adalah nama file, bukan display name
  model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
  )

  history = model.fit(
    x_train, y_train,
    batch_size=32,
    epochs=epochs, # Gunakan epochs dari parameter
    validation_data=(x_val, y_val),
    verbose=1
  )

  # Membuat direktori jika belum ada (dipindahkan ke awal skrip utama atau di sini per model)
  os.makedirs('models', exist_ok=True)
  os.makedirs('histories', exist_ok=True)
  os.makedirs('results/cnn_plots', exist_ok=True) # Pastikan direktori plot ada

  model.save(f'models/{model_name}.h5') # model_name adalah nama file

  with open(f'histories/{model_name}_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

  plot_and_save_single_history(history, model_name, "cnn_experiment") 

  return model, history

# Eksperimen Hyperparameter

In [19]:
# Pengaruh Jumlah Layer Konvolusi
os.makedirs('models', exist_ok=True)
os.makedirs('histories', exist_ok=True)

model_1_layer = create_cnn_model(conv_layers=1, filters_per_layer=[32])
model_1_layer, history_1_layer = train_model(model_1_layer, 'model_1_conv_layer')

model_2_layer = create_cnn_model(conv_layers=2, filters_per_layer=[32, 64])
model_2_layer, history_2_layer = train_model(model_2_layer, 'model_2_conv_layers')

model_3_layer = create_cnn_model(conv_layers=3, filters_per_layer=[32, 64, 128])
model_3_layer, history_3_layer = train_model(model_3_layer ,'model_3_conv_layers')

Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - accuracy: 0.3009 - loss: 1.8948 - val_accuracy: 0.5208 - val_loss: 1.3825
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 22s 18ms/step - accuracy: 0.4516 - loss: 1.4992 - val_accuracy: 0.5379 - val_loss: 1.3115


Saved plot to results/cnn_plots/cnn_experiment_model_1_conv_layer_history.png
Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.3045 - loss: 1.8738 - val_accuracy: 0.5363 - val_loss: 1.3156
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - accuracy: 0.4870 - loss: 1.4040 - val_accuracy: 0.5725 - val_loss: 1.2324


Saved plot to results/cnn_plots/cnn_experiment_model_2_conv_layers_history.png
Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.2815 - loss: 1.9204 - val_accuracy: 0.5178 - val_loss: 1.3321
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 17s 13ms/step - accuracy: 0.5174 - loss: 1.3503 - val_accuracy: 0.6032 - val_loss: 1.1265


Saved plot to results/cnn_plots/cnn_experiment_model_3_conv_layers_history.png


In [20]:
# Pengaruh Jumlah Filter per Layer
model_low_filters = create_cnn_model(conv_layers=2, filters_per_layer=[16, 32])
model_low_filters, history_low_layer = train_model(model_low_filters, 'model_low_filters')

model_med_filters = create_cnn_model(conv_layers=2, filters_per_layer=[32, 64])
model_med_filters, history_med_layer = train_model(model_med_filters, 'model_med_filters')

model_high_filters = create_cnn_model(conv_layers=2, filters_per_layer=[64, 128])
model_high_filters, history_high_layer = train_model(model_high_filters, 'model_high_filters')

Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.2828 - loss: 1.9350 - val_accuracy: 0.4969 - val_loss: 1.3947
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.4668 - loss: 1.4702 - val_accuracy: 0.5606 - val_loss: 1.2751


Saved plot to results/cnn_plots/cnn_experiment_model_low_filters_history.png
Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.2914 - loss: 1.9007 - val_accuracy: 0.5209 - val_loss: 1.3613
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - accuracy: 0.4933 - loss: 1.3925 - val_accuracy: 0.5951 - val_loss: 1.1559


Saved plot to results/cnn_plots/cnn_experiment_model_med_filters_history.png
Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 36s 28ms/step - accuracy: 0.3102 - loss: 1.8526 - val_accuracy: 0.5234 - val_loss: 1.3207
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 35s 28ms/step - accuracy: 0.5080 - loss: 1.3661 - val_accuracy: 0.6044 - val_loss: 1.1365


Saved plot to results/cnn_plots/cnn_experiment_model_high_filters_history.png


In [21]:
# Pengaruh Ukuran Filter
model_small_kernel = create_cnn_model(conv_layers=2, filters_per_layer=[32, 64], kernel_sizes=[3, 3])
model_small_kernel, history_small_kernel = train_model(model_small_kernel, 'model_small_kernel')

model_med_kernel = create_cnn_model(conv_layers=2, filters_per_layer=[32, 64], kernel_sizes=[5, 5])
model_med_kernel, history_med_kernel = train_model(model_med_kernel, 'model_med_kernel')

model_large_kernel = create_cnn_model(conv_layers=2, filters_per_layer=[32, 64], kernel_sizes=[7, 7])
model_large_kernel, history_large_kernel = train_model(model_large_kernel, 'model_large_kernel')

Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - accuracy: 0.3282 - loss: 1.8240 - val_accuracy: 0.5659 - val_loss: 1.2331
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - accuracy: 0.5395 - loss: 1.2959 - val_accuracy: 0.6171 - val_loss: 1.0868


Saved plot to results/cnn_plots/cnn_experiment_model_small_kernel_history.png
Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.2904 - loss: 1.9081 - val_accuracy: 0.5237 - val_loss: 1.3163
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 18s 14ms/step - accuracy: 0.4997 - loss: 1.3945 - val_accuracy: 0.5806 - val_loss: 1.1726


Saved plot to results/cnn_plots/cnn_experiment_model_med_kernel_history.png
Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 37s 29ms/step - accuracy: 0.2816 - loss: 1.9352 - val_accuracy: 0.4861 - val_loss: 1.4407
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 23s 19ms/step - accuracy: 0.4689 - loss: 1.4703 - val_accuracy: 0.5504 - val_loss: 1.2609


Saved plot to results/cnn_plots/cnn_experiment_model_large_kernel_history.png


In [22]:
# Pengaruh Jenis Pooling
model_max_pool = create_cnn_model(conv_layers=2, filters_per_layer=[32, 64], pooling_type='max')
model_max_pool, history_max_pool = train_model(model_max_pool, 'model_max_pooling')

model_avg_pool = create_cnn_model(conv_layers=2, filters_per_layer=[32, 64], pooling_type='avg')
model_avg_pool, history_avg_pool = train_model(model_avg_pool, 'model_avg_pooling')

Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - accuracy: 0.3011 - loss: 1.8899 - val_accuracy: 0.5427 - val_loss: 1.3078
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - accuracy: 0.4939 - loss: 1.3960 - val_accuracy: 0.5835 - val_loss: 1.1906


Saved plot to results/cnn_plots/cnn_experiment_model_max_pooling_history.png
Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.3082 - loss: 1.8735 - val_accuracy: 0.5084 - val_loss: 1.3864
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - accuracy: 0.4730 - loss: 1.4645 - val_accuracy: 0.5560 - val_loss: 1.2389


Saved plot to results/cnn_plots/cnn_experiment_model_avg_pooling_history.png


# Evaluasi dan Analisis

In [23]:
def evaluate_model(model, model_name):
  y_pred_proba = model.predict(x_test)
  y_pred = np.argmax(y_pred_proba, axis=1)

  f1_macro = f1_score(y_test, y_pred, average='macro')

  test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

  print(f"Model: {model_name}")
  print(f"Test Accuracy: {test_acc:.4f}")
  print(f"Macro F1-Score: {f1_macro:.4f}")
  print("-" * 40)

  return test_acc, f1_macro


# Implementasi Forward Propagation From Scratch

In [ ]:
from model import CNNFromScratch

# Testing dan Validasi

In [26]:
def test_forward_propagation(keras_model, model_name):
    """
    Test implementasi forward propagation from scratch
    """
    # Load model dari file
    keras_model = keras.models.load_model(f'models/{model_name}.h5')
    
    # Buat model from scratch
    scratch_model = CNNFromScratch(keras_model)
    
    # Test dengan subset data test (untuk efisiensi)
    test_samples = x_test[:100]
    
    # Prediksi dengan Keras
    keras_pred = keras_model.predict(test_samples)
    keras_pred_classes = np.argmax(keras_pred, axis=1)
    
    # Prediksi dengan implementasi from scratch
    scratch_pred = scratch_model.predict(test_samples)
    scratch_pred_classes = np.argmax(scratch_pred, axis=1)
    
    # Bandingkan hasil
    accuracy_match = np.mean(keras_pred_classes == scratch_pred_classes)
    
    # Hitung F1-score untuk kedua implementasi
    y_true = y_test[:100]
    keras_f1 = f1_score(y_true, keras_pred_classes, average='macro')
    scratch_f1 = f1_score(y_true, scratch_pred_classes, average='macro')
    
    print(f"Model: {model_name}")
    print(f"Prediction Match Accuracy: {accuracy_match:.4f}")
    print(f"Keras F1-Score: {keras_f1:.4f}")
    print(f"From Scratch F1-Score: {scratch_f1:.4f}")
    print(f"Difference in F1-Score: {abs(keras_f1 - scratch_f1):.6f}")
    print("-" * 50)
    
    return accuracy_match, keras_f1, scratch_f1

# Langkah Eksekusi Lengkap

In [27]:
def main():
    # 1. Persiapan data
    print("1. Loading and preprocessing CIFAR-10 data...")
    # (kode preprocessing di atas)
    
    # 2. Training semua variasi model
    print("2. Training models with different configurations...")
    # (kode training di atas)
    
    # 3. Evaluasi semua model
    print("3. Evaluating all models...")
    results = {}
    
    # Evaluasi untuk setiap eksperimen
    model_configs = [
        ('model_1_conv_layer', '1 Conv Layer'),
        ('model_2_conv_layers', '2 Conv Layers'),
        ('model_3_conv_layers', '3 Conv Layers'),
        ('model_low_filters', 'Low Filters'),
        ('model_med_filters', 'Medium Filters'),
        ('model_high_filters', 'High Filters'),
        ('model_small_kernel', 'Small Kernel'),
        ('model_med_kernel', 'Medium Kernel'),
        ('model_large_kernel', 'Large Kernel'),
        ('model_max_pooling', 'Max Pooling'),
        ('model_avg_pooling', 'Average Pooling')
    ]
    
    for model_name, display_name in model_configs:
        model = keras.models.load_model(f'models/{model_name}.h5')
        test_acc, f1_macro = evaluate_model(model, display_name)
        results[model_name] = {'accuracy': test_acc, 'f1_score': f1_macro}
    
    # 4. Plot perbandingan
    print("4. Plotting comparisons...")
    # (kode plotting di atas)
    
    # 5. Test forward propagation
    print("5. Testing forward propagation implementation...")
    for model_name, display_name in model_configs:
        test_forward_propagation(keras.models.load_model(f'models/{model_name}.h5'), model_name)
    
    # 6. Simpan hasil
    print("6. Saving results...")
    with open('experiment_results.pkl', 'wb') as f:
        pickle.dump(results, f)
    
    print("All experiments completed!")

if __name__ == "__main__":
    main()

1. Loading and preprocessing CIFAR-10 data...
2. Training models with different configurations...
3. Evaluating all models...
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


Model: 1 Conv Layer
Test Accuracy: 0.5403
Macro F1-Score: 0.5293
----------------------------------------
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


Model: 2 Conv Layers
Test Accuracy: 0.5777
Macro F1-Score: 0.5669
----------------------------------------
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


Model: 3 Conv Layers
Test Accuracy: 0.6077
Macro F1-Score: 0.6048
----------------------------------------
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


Model: Low Filters
Test Accuracy: 0.5650
Macro F1-Score: 0.5631
----------------------------------------
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


...

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


Model: model_small_kernel
Prediction Match Accuracy: 1.0000
Keras F1-Score: 0.6410
From Scratch F1-Score: 0.6410
Difference in F1-Score: 0.000000
--------------------------------------------------
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


Model: model_med_kernel
Prediction Match Accuracy: 1.0000
Keras F1-Score: 0.6592
From Scratch F1-Score: 0.6592
Difference in F1-Score: 0.000000
--------------------------------------------------
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


Model: model_large_kernel
Prediction Match Accuracy: 1.0000
Keras F1-Score: 0.5986
From Scratch F1-Score: 0.5986
Difference in F1-Score: 0.000000
--------------------------------------------------
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


Model: model_max_pooling
Prediction Match Accuracy: 1.0000
Keras F1-Score: 0.5933
From Scratch F1-Score: 0.5933
Difference in F1-Score: 0.000000
--------------------------------------------------
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Model: model_avg_pooling
Prediction Match Accuracy: 1.0000
Keras F1-Score: 0.6462
From Scratch F1-Score: 0.6462
Difference in F1-Score: 0.000000
--------------------------------------------------
6. Saving results...
All experiments completed!
